In [1]:
# Importing Librabries and Setting options
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_info_columns',100)
pd.set_option('display.max_info_rows',100)

# Reading Input data loan.csv

In [2]:
loan_raw = pd.read_csv("loan.csv",low_memory=False)

In [3]:
loan_raw.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [4]:
loan_raw.shape

(39717, 111)

# Data Cleansing

In [5]:
# Checking columns which has all NULLS in loan.csv
loan_raw.isnull().sum()

id                                0
member_id                         0
loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
                              ...  
tax_liens                        39
tot_hi_cred_lim               39717
total_bal_ex_mort             39717
total_bc_limit                39717
total_il_high_credit_limit    39717
Length: 111, dtype: int64

In [6]:
# 1st Step : Removing columns which has all NULLS in loan.csv
loan =loan_raw.dropna(axis=1,how='all')

In [7]:
loan.shape

(39717, 57)

In [8]:
loan.isnull().sum()

id                                0
member_id                         0
loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
term                              0
int_rate                          0
installment                       0
grade                             0
sub_grade                         0
emp_title                      2459
emp_length                     1075
home_ownership                    0
annual_inc                        0
verification_status               0
issue_d                           0
loan_status                       0
pymnt_plan                        0
url                               0
desc                          12940
purpose                           0
title                            11
zip_code                          0
addr_state                        0
dti                               0
delinq_2yrs                       0
earliest_cr_line                  0
inq_last_6mths              

In [9]:
# Finding those columns who are missing in between 15-30%
# We should remove such variables as they does not have good fill rate
for col in loan:
    if (loan[col].count()/len(loan)) <0.30:
        print(col)

mths_since_last_record
next_pymnt_d


In [10]:
# Step 2: Removing such variables : mths_since_last_record,next_pymnt_d
list_not_good_fill_rate = ['mths_since_last_record','next_pymnt_d']
loan.drop(list_not_good_fill_rate,axis=1, inplace= True)

In [11]:
# customer behaviour and few other variables which should not be having any influence
# Step 3 : We should remove them too
cust_behaviour_list = ['delinq_2yrs','earliest_cr_line','inq_last_6mths','open_acc','pub_rec','revol_bal','revol_util','total_acc','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt','last_credit_pull_d','application_type','desc','url','id','member_id','zip_code']
loan.drop(cust_behaviour_list, axis=1, inplace= True)

In [12]:
loan.shape

(39717, 29)

In [13]:
# Data type of variable that we need to analyse
loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 29 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   loan_amnt                   int64  
 1   funded_amnt                 int64  
 2   funded_amnt_inv             float64
 3   term                        object 
 4   int_rate                    object 
 5   installment                 float64
 6   grade                       object 
 7   sub_grade                   object 
 8   emp_title                   object 
 9   emp_length                  object 
 10  home_ownership              object 
 11  annual_inc                  float64
 12  verification_status         object 
 13  issue_d                     object 
 14  loan_status                 object 
 15  pymnt_plan                  object 
 16  purpose                     object 
 17  title                       object 
 18  addr_state                  object 
 19  dti                      

In [14]:
(loan.term.value_counts()*100)/len(loan)

 36 months    73.258302
 60 months    26.741698
Name: term, dtype: float64

In [15]:
(loan.int_rate.value_counts()*100)/len(loan)

10.99%    2.407030
13.49%    2.079714
11.49%    2.077196
7.51%     1.981519
7.88%     1.825415
            ...   
18.36%    0.002518
16.96%    0.002518
16.15%    0.002518
16.01%    0.002518
17.44%    0.002518
Name: int_rate, Length: 371, dtype: float64

In [16]:
(loan.emp_length.value_counts()*100)/len(loan)

10+ years    22.355666
< 1 year     11.539139
2 years      11.048166
3 years      10.310446
4 years       8.651207
5 years       8.263464
1 year        8.157716
6 years       5.612206
7 years       4.464083
8 years       3.723846
9 years       3.167409
Name: emp_length, dtype: float64

In [17]:
# Employment Length is having NA in the string , we should fill it
loan['emp_length'] = loan['emp_length'].fillna('0 Year')

In [18]:
(loan.emp_length.value_counts()*100)/len(loan)

10+ years    22.355666
< 1 year     11.539139
2 years      11.048166
3 years      10.310446
4 years       8.651207
5 years       8.263464
1 year        8.157716
6 years       5.612206
7 years       4.464083
8 years       3.723846
9 years       3.167409
0 Year        2.706650
Name: emp_length, dtype: float64

In [19]:
# Interest Rate is having % in the string , we should remove it and make it Numeric data type
loan['int_rate']= loan['int_rate'].str.replace('%','')
loan['int_rate']= pd.to_numeric(loan['int_rate'])

In [20]:
loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 29 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   loan_amnt                   int64  
 1   funded_amnt                 int64  
 2   funded_amnt_inv             float64
 3   term                        object 
 4   int_rate                    float64
 5   installment                 float64
 6   grade                       object 
 7   sub_grade                   object 
 8   emp_title                   object 
 9   emp_length                  object 
 10  home_ownership              object 
 11  annual_inc                  float64
 12  verification_status         object 
 13  issue_d                     object 
 14  loan_status                 object 
 15  pymnt_plan                  object 
 16  purpose                     object 
 17  title                       object 
 18  addr_state                  object 
 19  dti                      

In [21]:
loan['emp_title'] = loan['emp_title'].fillna('Unknown')

Unknown                            6.191303
US Army                            0.337387
Bank of America                    0.274442
IBM                                0.166176
AT&T                               0.148551
                                     ...   
AMEC                               0.002518
lee county sheriff                 0.002518
Bacon County Board of Education    0.002518
Hotpads                            0.002518
Evergreen Center                   0.002518
Name: emp_title, Length: 28821, dtype: float64

In [22]:
loan[loan['emp_title']=='Unknown']

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,dti,mths_since_last_delinq,initial_list_status,collections_12_mths_ex_med,policy_code,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000,5000,4975.0,36 months,10.65,162.87,B,B2,Unknown,10+ years,...,27.65,NaN,f,0.0,1,0,0.0,0,0.0,0.0
2,2400,2400,2400.0,36 months,15.96,84.33,C,C5,Unknown,10+ years,...,8.72,NaN,f,0.0,1,0,0.0,0,0.0,0.0
8,5600,5600,5600.0,60 months,21.28,152.39,F,F2,Unknown,4 years,...,5.55,NaN,f,0.0,1,0,0.0,0,0.0,0.0
30,4375,4375,4375.0,36 months,7.51,136.11,A,A3,Unknown,7 years,...,20.34,NaN,f,0.0,1,0,0.0,0,0.0,0.0
42,3000,3000,3000.0,36 months,18.25,108.84,D,D5,Unknown,9 years,...,17.39,NaN,f,0.0,1,0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39700,5400,5400,200.0,36 months,7.75,168.60,A,A3,Unknown,< 1 year,...,3.00,0.0,f,NaN,1,0,NaN,0,NaN,NaN
39704,3000,3000,950.0,36 months,8.70,94.98,B,B1,Unknown,< 1 year,...,14.54,0.0,f,NaN,1,0,NaN,0,NaN,NaN
39706,5000,5000,150.0,36 months,8.07,156.84,A,A4,Unknown,< 1 year,...,5.55,0.0,f,NaN,1,0,NaN,0,NaN,NaN
39714,5000,5000,1325.0,36 months,8.07,156.84,A,A4,Unknown,< 1 year,...,2.30,0.0,f,NaN,1,0,NaN,0,NaN,NaN


In [23]:
loan.isnull().sum()

loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
term                              0
int_rate                          0
installment                       0
grade                             0
sub_grade                         0
emp_title                         0
emp_length                        0
home_ownership                    0
annual_inc                        0
verification_status               0
issue_d                           0
loan_status                       0
pymnt_plan                        0
purpose                           0
title                            11
addr_state                        0
dti                               0
mths_since_last_delinq        25682
initial_list_status               0
collections_12_mths_ex_med       56
policy_code                       0
acc_now_delinq                    0
chargeoff_within_12_mths         56
delinq_amnt                       0
pub_rec_bankruptcies        

In [24]:
(loan.emp_title.value_counts()*100)/len(loan)

Unknown                            6.191303
US Army                            0.337387
Bank of America                    0.274442
IBM                                0.166176
AT&T                               0.148551
                                     ...   
AMEC                               0.002518
lee county sheriff                 0.002518
Bacon County Board of Education    0.002518
Hotpads                            0.002518
Evergreen Center                   0.002518
Name: emp_title, Length: 28821, dtype: float64

In [27]:
loan['issue_d']

0        Dec-11
1        Dec-11
2        Dec-11
3        Dec-11
4        Dec-11
          ...  
39712    Jul-07
39713    Jul-07
39714    Jul-07
39715    Jul-07
39716    Jun-07
Name: issue_d, Length: 39717, dtype: object

In [28]:
(loan.purpose.value_counts()*100)/len(loan)

debt_consolidation    46.934562
credit_card           12.916383
other                 10.053629
home_improvement       7.493013
major_purchase         5.506458
small_business         4.602563
car                    3.900093
wedding                2.384369
medical                1.744845
moving                 1.467885
vacation               0.959287
house                  0.959287
educational            0.818289
renewable_energy       0.259335
Name: purpose, dtype: float64

In [29]:
duplicate = loan[loan.duplicated()]

In [30]:
duplicate.shape

(0, 29)

In [ ]:
# List which we discussed should not be in final list - policy_code(It has all 1),collections_12_mths_ex_med(Very low fill rate)
# initial_list_status (only have f),acc_now_delinq(It has only 0), chargeoff_within_12_mths (It has only 0 and NA), delinq_amnt(all 0)

In [31]:
loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 29 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   loan_amnt                   int64  
 1   funded_amnt                 int64  
 2   funded_amnt_inv             float64
 3   term                        object 
 4   int_rate                    float64
 5   installment                 float64
 6   grade                       object 
 7   sub_grade                   object 
 8   emp_title                   object 
 9   emp_length                  object 
 10  home_ownership              object 
 11  annual_inc                  float64
 12  verification_status         object 
 13  issue_d                     object 
 14  loan_status                 object 
 15  pymnt_plan                  object 
 16  purpose                     object 
 17  title                       object 
 18  addr_state                  object 
 19  dti                      

In [32]:
(loan.pub_rec_bankruptcies.value_counts()*100)/len(loan)

0.0    94.012639
1.0     4.214820
2.0     0.017625
Name: pub_rec_bankruptcies, dtype: float64

In [33]:
(loan.loan_status.value_counts()*100)/len(loan)

Fully Paid     82.961956
Charged Off    14.167737
Current         2.870307
Name: loan_status, dtype: float64

In [25]:
(loan.loan_status.value_counts()*100)/len(loan)

Fully Paid     82.961956
Charged Off    14.167737
Current         2.870307
Name: loan_status, dtype: float64

In [26]:
# customer behaviour